In [79]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import random
import pickle

In [80]:
DataDir = "C:/Users/M A H D I/Desktop/Jupyter_W/DataSet1"
Categories = ["Defected","OK"]
IMG_SIZE = 256
data = []

def create_datslist():
    for category in Categories:
        path = os.path.join(DataDir,category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                img_N = cv2.normalize(img_array,None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                new_array = cv2.resize(img_N, (IMG_SIZE, IMG_SIZE))
                data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_datslist()


In [81]:
x =[]
y= []

for features, label in data:
    x.append(features)
    y.append(label)

x = np.array(x)
y = np.array(y)


In [82]:
print(np.unique(y, return_counts=True))

(array([0, 1]), array([750, 500], dtype=int64))


In [83]:
X = x.reshape(-1,IMG_SIZE*IMG_SIZE)
print(X.shape)


(1250, 65536)


# KNN

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

clf = KNeighborsClassifier(n_neighbors=5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6, stratify=y)

clf.fit(X_train, y_train)      # train
y_pred = clf.predict(X_test)   # test


test_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data = %.2f" % (test_accuracy * 100) + " % \n")

print(np.unique(y_test, return_counts=True),'\n')
cm = confusion_matrix(y_test, y_pred)
print(cm)


Accuracy on test data = 87.60 % 

(array([0, 1]), array([150, 100], dtype=int64)) 

[[134  16]
 [ 15  85]]


# CNN

In [85]:
import tensorflow as tf
from tensorflow import keras


In [86]:
x = x.reshape(-1,IMG_SIZE,IMG_SIZE,1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10, stratify=y)

In [106]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(64, (3,3), input_shape=(256,256,1), padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D())

model.add(keras.layers.Conv2D(50, (3,3),padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D())

model.add(keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(98, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(27, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(2, activation='softmax'))

In [107]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [108]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 256, 256, 64)      640       
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 128, 128, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_44 (Conv2D)          (None, 128, 128, 50)      28850     
                                                                 
 max_pooling2d_44 (MaxPooli  (None, 64, 64, 50)        0         
 ng2D)                                                           
                                                                 
 conv2d_45 (Conv2D)          (None, 64, 64, 32)        14432     
                                                                 
 max_pooling2d_45 (MaxPooli  (None, 32, 32, 32)      

In [114]:
weights_filename = 'CNN-weights_84.hs'

if os.path.exists(weights_filename):
    model.load_weights(weights_filename)

In [115]:
his = model.fit(x_train, y_train, epochs=5, batch_size=8, validation_data=(x_test, y_test))

Epoch 1/5
125/125 [==============================] - 81s 646ms/step - loss: 0.2064 - accuracy: 0.8900 - val_loss: 0.3132 - val_accuracy: 0.8280
Epoch 2/5
125/125 [==============================] - 85s 679ms/step - loss: 0.1697 - accuracy: 0.9170 - val_loss: 0.3319 - val_accuracy: 0.8520
Epoch 3/5
125/125 [==============================] - 82s 655ms/step - loss: 0.1926 - accuracy: 0.9190 - val_loss: 0.3756 - val_accuracy: 0.8640
Epoch 4/5
125/125 [==============================] - 78s 623ms/step - loss: 0.1767 - accuracy: 0.9230 - val_loss: 0.3123 - val_accuracy: 0.8480
Epoch 5/5
125/125 [==============================] - 84s 672ms/step - loss: 0.1220 - accuracy: 0.9420 - val_loss: 0.3048 - val_accuracy: 0.8760


In [116]:
model.save_weights('CNN-weights_84.hs')